In [3]:
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-lv-60-espeak-cv-ft")
# processor = Wav2Vec2Processor.from_pretrained(
#     "facebook/wav2vec2-lv-60-espeak-cv-ft")

# processor.save_pretrained("model")
# model.save_pretrained("model")


In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained(
    "model")
model = Wav2Vec2ForCTC.from_pretrained(
    "model")


2023-03-20 21:18:56.211063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 21:18:56.306822: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import json
import secrets

import sagemaker
from datasets import Audio, Dataset, interleave_datasets
from model.code.inference import input_fn, model_fn, predict_fn
from pydub import AudioSegment

model_dir = "model"
print(model_dir)

model = model_fn(model_dir)

# ogg_audio = '/root/ws/ml/voicematch-backend/dwl/hello-hello.ogg'
ogg_audio = '/root/ws/ml/voicematch-backend/dwl/youtube/horMQDOeqZ0-4_5.ogg' # worker died

with open(ogg_audio, 'rb') as f:
    audio_data = f.read()
print(audio_data)

input_data = input_fn(audio_data, "audio/ogg")
print('input_data: ', input_data)

phonemes = predict_fn(input_data, model)
print('phonemes: ', phonemes)

# output_data = output_fn(words, "application/json")
# print('output_data: ', words)

# output = json.dumps(words)
# print('output: ', output)

model
loading model from model
b'OggS\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00?\xebe\xe5\x00\x00\x00\x00e\x9d\xb1\x9d\x01\x1e\x01vorbis\x00\x00\x00\x00\x02D\xac\x00\x00\x00\x00\x00\x00\x80\xb5\x01\x00\x00\x00\x00\x00\xb8\x01OggS\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00?\xebe\xe5\x01\x00\x00\x00\xb5/\x1c\x82\x11\xd2\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x07\x03vorbis\r\x00\x00\x00Lavf58.29.100\x06\x00\x00\x00\x0c\x00\x00\x00language=eng3\x00\x00\x00handler_name=ISO Media file produced by Google Inc.\x1f\x00\x00\x00encoder=Lavc58.54.100 libvorbis\x10\x00\x00\x00major_brand=isom\x11\x00\x00\x00minor_version=512\x1e\x00\x00\x00compatible_brands=isomiso2mp41\x01\x05vorbis%BCV\x01\x00@\x00\x00$s\x18*F\xa5s\x16\x84\x10\x1aBP\x19\xe3\x1cB\xcek\xec\x19BL\x11\x82\x1c2L[\xcb%s\x90!\xa4\xa0B\x88[(\x81\xd0\x90U\x00\x00@\x00\x00\x87Ax\x14\x84\x8aA\x08!\x84%=X\x92\x83\'=\x08!\x84\x889x\x14\x84iA\x08!\x84\x10B\x08!\x84\x10B\x08!\x84E9h\x92\x83\'A\x08\x1d\x84\xe308\x0c\x83\xe58\xf8\

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


input_data:  {'path': '/tmp/vm-4e2771946c86e41bc2071820b3377be0f754d61f5c4e2479.wav', 'array': array([-0.01371232,  0.01298687,  0.03979859, ...,  0.11772479,
        0.09349819,  0.06682537], dtype=float32), 'sampling_rate': 16000}
predicting phonemes from /tmp/vm-4e2771946c86e41bc2071820b3377be0f754d61f5c4e2479.wav


: 

: 

In [10]:
# %cd /root/ws/ml/dlc/voicematch-dlc

# %rm -rf "model/"
# %mkdir "model/"

# !rsync -av --exclude '__pycache__' src/ model/code
!du -sh "model"

# excluding the model with configs as it's very heavy with intention to load dynamically for inference and cache
# !tar zcvf model-v3.tar.gz --exclude='__pycache__' --exclude='__init__.py'  --exclude='pytorch_model.bin'  --exclude='config.json' --exclude='./model.tar.gz' .
# %mv "model/pytorch_model.bin" "model/voicematch.pt"
!tar zcvf build/phonerecog.tar.gz --exclude='__pycache__' --exclude='__init__.py' --exclude='pytorch_model.bin' -C 'model' .

!du -sh build/phonerecog.tar.gz

# !tar zcvf "voicematch-bin.tar.gz" --exclude='__pycache__' --exclude='__init__.py' -C './model-bin' .


361M	models/wordrecog/model
./
./tokenizer_config.json
./vocab.json
./config.json
./code/
./code/inference.py
./preprocessor_config.json
./special_tokens_map.json
4.0K	build/voicematch-model.tar.gz


In [1]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

# try:
#     role = sagemaker.get_execution_role()
# except ValueError:
#     iam = boto3.client('iam')
#     role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

role = "arn:aws:iam::908060038426:role/sagemaker-local-role"

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker role arn: arn:aws:iam::908060038426:role/sagemaker-local-role
sagemaker bucket: sagemaker-us-east-1-908060038426
sagemaker session region: us-east-1


In [2]:
model_s3_path = sess.upload_data(
    path="build/phonerecog.tar.gz", bucket=sess.default_bucket(), key_prefix="model/voicematch"
)
print(f"model_s3_path: {model_s3_path}")

model_s3_path: s3://sagemaker-us-east-1-908060038426/model/voicematch/voicematch-model.tar.gz


In [12]:
%load_ext autoreload
%autoreload all

from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.pytorch import PyTorchModel
from src.predictor import PlainPyTorchPredictor

# print(f"model_s3_path: {model_s3_path}")
role = "arn:aws:iam::908060038426:role/sagemaker-local-role"
model_s3_path = "s3://sagemaker-us-east-1-908060038426/model/voicematch/phonerecog.tar.gz"

# authenticate to ECR
# !aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com
# !docker pull 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117

# model = HuggingFaceModel(
#    model_data=model_s3_path,
#    role=role,
# #    transformers_version="4.12",
# #    pytorch_version="1.9",
# #    py_version='py38',
#    image_uri="763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117",
# )

model = PyTorchModel(
    entry_point="inference.py",
    source_dir="model/code",
    role=role,
    model_data=model_s3_path,
    model_server_workers=4,
    image_uri='voicematch-phonerecog:latest',
    # predictor_cls=PlainPyTorchPredictor
)

from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import StreamDeserializer

# !docker rm $(docker stop $(docker ps -aq --filter ancestor=voicematch-wordrecog:latest --format="{{.ID}}"))
# predictor.delete_endpoint()

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="local",
    serializer=IdentitySerializer(),
    deserializer=StreamDeserializer(),
    # wait=False,
)
print(f"predictor: {predictor}")
print(f"predictor.endpoint_name: {predictor.endpoint_name}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Attaching to wbs0vskpmy-algo-1-5m0wn
wbs0vskpmy-algo-1-5m0wn | Exported: /opt/ml/model/pytorch_model.bin
wbs0vskpmy-algo-1-5m0wn | ['torchserve', '--start', '--model-store', '/.sagemaker/ts/models', '--ts-config', '/etc/sagemaker-ts.properties', '--log-config', '/opt/conda/lib/python3.9/site-packages/sagemaker_pytorch_serving_container/etc/log4j2.xml', '--models', 'model=/opt/ml/model']
wbs0vskpmy-algo-1-5m0wn | WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.
wbs0vskpmy-algo-1-5m0wn | Warning: Nashorn engine is planned to be removed from a future JDK release
wbs0vskpmy-algo-1-5m0wn | Warning: Nashorn engine is planned to be removed from a future JDK release
wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:18,396 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:18,591 [IN

In [13]:
from datasets import Audio, Dataset, interleave_datasets
from pydub import AudioSegment

print(predictor)

ogg_audio = '../../data/hello-hello.ogg'
with open(ogg_audio, 'rb') as f:
    audio_data = f.read()

# res = predictor.predict(audio_data, initial_args={
#     "ContentType": "audio/ogg",
# })
# predictor.content_type = "audio/ogg"
result = predictor.predict(audio_data)

print('len: ', len(result))
if len(result) > 0:
    print(f"result[0]: {result[0]}")
if len(result) > 1:
    print(f"result[1]: {result[1]}")

if len(result) > 0:
    response = result[0]
    status = response.status
    content_type = response.headers['Content-Type']
    data = response.read()

    print(f"status: {status}")
    print(f"content_type: {content_type}")
    print(f"data: {data}")


wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:51,178 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Flushing req. to backend at: 1678338171178
wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:51,180 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - running model_fn
wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:51,180 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Backend received inference at: 1678338171
wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:55,339 [WARN ] W-9000-model_1.0-stderr MODEL_LOG - It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:55,643 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - running input_fn, content_type: application/octet-stream
wbs0vskpmy-algo-1-5m0wn | 2023-03-09T05:02:55,643 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - ogg_file: /tmp/vm-1286b1fc44ff2d9a0609f0edc1da18b9d629943b4563e5ff.ogg
wbs0vskpmy-algo-1-5m0wn | 2023

In [11]:
# delete endpoint
predictor.delete_endpoint()


Gracefully stopping... (press Ctrl+C again to force)
